In this notebook, I'll provide an wide overview on Automatic Speech Architecture, its standard pipelines, and how to train a model for it!

### What is Automatic Speech Recognition?
Automatic Speech Recognition (ASR) is the translation of spoken speech to text by a computer.

It has many uses:
+ closed captioning
+ mobile phone voice assistants
+ interface for handicapped individuals
+ preserve endangered languages

However, ASR is very difficult to solve due to:
+ Environment (Is there noise? Other speakers?)
+ Style of speech (Is it casual, formal, poetic, etc?)
+ Style of speaker (Talks fast? Accents? Gender bias?)

https://www.youtube.com/watch?v=q67z7PTGRi8

Modern ASR models work quite well in ideal, noiseless conditions. However, in noisy conditions they are far from perfect. Below, we can see the typical accuracy for state-of-the-art ASR models in 2020!

(Note: To calculate the accuracy of ASR models, we use a 'Word Error Rate.' This determines which percentage of spoken words were incorrectly translated to text by the ASR model. )

![word error rate](https://drive.google.com/uc?export=view&id=1iUz9koIqQErBVB0a4db0gSZVdfsfwh9L)



Architecture:

![timeline](https://drive.google.com/uc?export=view&id=1n2ID2ML5x9TFSBJhfdvj5AGI4u-9Mp4m)


From the 1980s to the 2010s, ASR was predominately performed by Hidden Markov Model (HMM) architecture. HMMs are probabilistic models for linear sequence classification problems. Given text's linear nature, HMMs seem like a natural fit for ASR.
However, using HMMs for ASR come with 3 downsides:
+ HMMs are built on the Markov Assumption, which assumes that the probability of the next state only depends on the current state, and not any states prior. This doesn't make sense for language processing, which is highly contextual (see Coarticulation)
+ HMMs' state transitions probabilities are 'baked in' and thus inflexible to changes in language
+ Classic HMM ASR pipelines require hand-tuned probability distributions for accoustic models and language models from linguistic experts
+ HMM-based ASR architecture is very complex, requiring 3 models as input. The figure below demonstrates an HMM-centric ASR architecture

![an HMM-based architecture](https://drive.google.com/uc?export=view&id=1loF8wbD-6DRO45Ly7lKnVVIcCn83H-B9)

Nowadays Recurrent Neural Networks (RNNs) or RNN-HMM hybrids are favored for ASR because they solve these problems. However, RNN's come at the cost of requiring a vast amount of training data.

![an RNN-only architecture](https://drive.google.com/uc?export=view&id=1_9McwMlHIqlPuJGYOkPwB-8h-t13P4pI)
(an RNN-only model that has only 4 steps)

https://stats.stackexchange.com/questions/282987/hidden-markov-model-vs-recurrent-neural-network
Jurafsky textbook

For the our ASR model, we'll be using an 'Listen, Attend, and Spell' (LAS) RNN. The LAS was proposed by in 2016 for ASR architecture in order to simplify Hybrid architecture (such as DNN-HMMs or CTC-HMMs) and avoid the aforementioned independence assumptions. Since then it has been the forefront of ASR architecture.

https://storage.googleapis.com/pub-tools-public-publication-data/pdf/44926.pdf

The objective of LAS is to transform an arbitrary-length input sequence to an arbitrary-length output sequence. LAS is an encoder-decoder model- a class of RNN's that is well-suited for problems where the length of the inputs greatly differs from the length of the outputs (like ASR).

Encoder-Decoder models are composed of 3 parts:
+ an Encoder
+ a Context Vector
+ a Decoder

The Encoder compresses and summarizes input data into a Context Vector, which is later transformed by the Decoder. The Context Vector must be defined with a fixed-length (usually 256, 512, or 1024).

Accoustic Information as Input for RNN:
A classic HMM-based ASR model would require us to input 3 probabilistic models (accoustic, pronunciation, and language), whereas an RNN only requires 1 (accoustic). The accoustic model captures important characteristics about how we perceive speech (see MFCC). We'll extract this data from the LibriSpeech ASR dataset.

https://www.openslr.org/12

We need to first extract features from the audio files- we'll be putting these features in a vector and feeding them into the RNN.
In order to extract the features from a given file, we need to perform the following steps:
```
1) Convert the FLAC file into a Waveform (giving us frequency data over the time domain)
2) Split the Waveform into Windows using a Windowing Function (like Hamming)
3) For each Window:
    a) Compute its Fourier Transform
    b) Compute its Mel Spectrogram (also known as Mel Filter Bank)
    c) Compute the Log of each value in the Mel Spectrogram
    d) Append the result of 3c to the Features Vector
4) Return Features Vector
```

But before we do any computations, let's define some configuration constants:

In [192]:
sample_rate = 16000  # The Sample Rate of the LibriSpeech Dataset
window_size_frames = int(sample_rate * 0.025)  # The standard window size for ASR models is 25ms
window_step_frames = int(sample_rate * 0.010) # The standard window step for ASR models is 10ms
n_filters = 40

And of course, download the dataset(s):

In [17]:
DATASET_DIR = './data/'

DATASETS = [
  "dev-clean"
  # "train-clean-100"
  # "train-clean-360"  # this exceeds Colab's Disk limit
  # "train-clean-500"  # this exceeds Colab's Disk limit
]

def download_datasets(datasets): 
  for dataset in datasets:
    dest = DATASET_DIR + dataset + '.tar.gz'
    !mkdir -p {DATASET_DIR}
    !wget -O {dest} https://www.openslr.org/resources/12/{dataset}.tar.gz

def extract_datasets(datasets):
  for dataset in datasets:
    src = DATASET_DIR + dataset + ".tar.gz"
    dest = DATASET_DIR + dataset + '/'
    !mkdir -p {DATASET_DIR + dataset}
    !tar -xf {src} -C {dest}
    print("Finished extraction")

download_datasets(DATASETS)
extract_datasets(DATASETS)

--2021-06-06 20:02:27--  https://www.openslr.org/resources/12/dev-clean.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 337926286 (322M) [application/x-gzip]
Saving to: ‘./data/dev-clean.tar.gz’

./data/dev-clean.ta 100%[===================>] 322.27M  25.9MB/s    in 13s     

2021-06-06 20:02:41 (24.1 MB/s) - ‘./data/dev-clean.tar.gz’ saved [337926286/337926286]

Finished extraction


Now let's get the Log Mel Filter Bank energies! We'll be using the speechpy library for simplicity.

In [ ]:
import numpy as np
import soundfile as sf
import tensorflow as tf
import librosa
!pip install speechpy
import speechpy

In [214]:

def files_to_log_mel_spec(paths, sample_rate, window_size_frames, window_step_frames, num_mel_bins):
  """ Returns 
  ( Tensor of Log Mel Spectrograms of the audio files in paths, 
    numpy array of the Lengths of the Spectrograms ) 
  """
  lmspecs = [] 
  lengths = []
  for path in paths:
    audio = tf.io.read_file(path)
    audio, _ = librosa.load(path, sample_rate)
    audio = tf.convert_to_tensor(audio)
    # Get Spectrogram using Short-time Fourier Transform
    stfts = tf.signal.stft(audio, frame_length=window_size_frames, frame_step=window_step_frames, fft_length=1024)
    spectrograms = tf.abs(stfts)
    # Warp the linear scale spectrograms into the Mel Scale.
    num_spectrogram_bins = stfts.shape[-1]
    lower_edge_hertz, upper_edge_hertz = 80.0, 7600.0
    linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
      num_mel_bins, num_spectrogram_bins, sample_rate, lower_edge_hertz,
      upper_edge_hertz)
    mel_spectrograms = tf.tensordot(spectrograms, linear_to_mel_weight_matrix, 1)
    mel_spectrograms.set_shape(spectrograms.shape[:-1].concatenate(
      linear_to_mel_weight_matrix.shape[-1:]))
    # Compute a stabilized log to get log-magnitude mel-scale spectrograms.
    lmspec = tf.math.log(mel_spectrograms + 1e-6)
    lmspecs.append(lmspec)
    lengths.append(len(lmspec))
  # Normalize all
  means = tf.math.reduce_mean(lmspecs, 1, keepdims=True)
  stddevs = tf.math.reduce_std(lmspecs, 1, keepdims=True)
  lmspecs = (lmspecs - means) / stddevs
  audio_len = tf.shape(lmspecs)[0]
  # padding to 10 seconds
  # pad_len = 2754
  # paddings = tf.constant([[0, pad_len], [0, 0]])
  # lmspec = tf.pad(lmspec, paddings, "CONSTANT")[:pad_len, :]
  return lmspecs, np.array(lengths).astype(np.int32)

files_to_log_mel_spec(['./data/dev-clean/LibriSpeech/dev-clean/1272/128104/1272-128104-0000.flac'], sample_rate, window_size_frames, window_step_frames, n_filters)

# def files_to_log_mel_spec(filepaths, sample_rate, window_size, window_step, n_filters):  # LibriSpeech uses a 16kHz sample rate
#     """ Convert an audio file to a Log Mel Spectrogram """
#     log_mels = []
#     lengths = []
#     for filepath in filepaths:
#         test_sig, _ = librosa.load(filepath, sample_rate)
#         # Perform Mel Spec
#         # mel_spec = librosa.feature.melspectrogram(test_sig, sr=sample_rate, n_fft=window_size, hop_length=window_step, window='hann')
#         # log_mel = np.log(mel_spec)
#         log_mel = speechpy.feature.lmfe(test_sig, sample_rate, frame_length=0.025, frame_stride=0.010, num_filters=n_filters).astype(np.float32)
#         # log_mel = np.asarray(log_mel, dtype=np.float)
#         # print(log_mel.shape)
#         # Log it
#         log_mels.append(log_mel)
#         lengths.append(len(log_mels))
#     # print(log_mels)
#     log_mels = np.array(log_mels)
#     # print(log_mels.shape)
#     # normalize it
#     means = tf.math.reduce_mean(log_mels, 1, keepdims=True)
#     stddevs = tf.math.reduce_std(log_mels, 1, keepdims=True)
#     # print(log_mels)
#     # try:
#     #   means = np.mean(log_mels,axis=1,keepdims=True)  
#     # except AxisError:
#     #   print(log_mels.shape)
#     # stddevs = np.std(log_mels,axis=1,keepdims=True)
#     normalized = (log_mels - means) / stddevs
#     return normalized, np.array(lengths).astype(np.int32)

# Example
# files_to_log_mel_spec(['././data/dev-clean/LibriSpeech/dev-clean/1272/128104/1272-128104-0000.flac'], sample_rate, window_size, window_step, n_filters)
# test = np.array((1,2))
# tf.reduce_mean(test)

(<tf.Tensor: shape=(1, 584, 40), dtype=float32, numpy=
 array([[[-1.5120794 , -1.6673089 , -1.8040851 , ..., -1.566706  ,
          -1.4199985 , -1.4137021 ],
         [-1.6922767 , -1.7004411 , -1.5005974 , ..., -1.4650229 ,
          -1.4228181 , -1.3736689 ],
         [-1.3246887 , -1.3394393 , -1.3179259 , ..., -1.448277  ,
          -1.4855719 , -1.4314622 ],
         ...,
         [-1.558339  , -1.6693529 , -1.4325359 , ..., -1.3655134 ,
          -1.0403051 , -0.98255634],
         [-1.4902389 , -1.6166751 , -1.520029  , ..., -1.36802   ,
          -0.9872307 , -0.9673391 ],
         [-1.516491  , -1.4677212 , -1.3451515 , ..., -1.5840408 ,
          -0.93954355, -0.90887135]]], dtype=float32)>,
 array([584], dtype=int32))

Sometimes you may see MFCCs used as the ASR features instead of Mel Filter Banks. MFCCs are values that loosely represents the brain's capacity to filter out certain signals. MFCCs were popular with HMM-based models for helping model based on human-perceptible information, but newer architecture can usually use Filter Banks and get similar or better accuracy. You can read more [here](https://haythamfayek.com/2016/04/21/speech-processing-for-machine-learning.html).

The diagram below shows the steps required for calculating the Filter Banks and MFCCs. As you can see, MFCCs require a few more calculations after the Filter Banks:

![features pipeline](https://drive.google.com/uc?export=view&id=15T0MgXcj9wA_ZQ_i4tCLukUCH-7T2FnB)

https://www.youtube.com/watch?v=QTw-6GU5Mjs&t=319s


Let's also add a helper function that'll allow us to map the spectrograms to their target text! Specifically, this function will Tokenize the target text (i.e. split the text by character) and Numericize it (i.e. map each character to an integer). We'll also insert some special boundary characters to help the NN read the string:

In [40]:
# This tokenization code was borrowed/modified from https://github.com/30stomercury/Automatic-Speech-Recognition/blob/master/utils/tokenizer.py
import string
def char2id(special_tokens):
    """
    Args:
        special_tokens: special charactors, <PAD>, <SOS>, <EOS>, <SPACE>
    Returns:
        char2id: dict, from character to index.
        id2char: dict, from index to character.
    """
    alphas = list(string.ascii_uppercase[:26])
    tokens = special_tokens + alphas
    token_to_id = {}
    id_to_token = {}
    for i, c in enumerate(tokens):
        token_to_id[c] = i
        id_to_token[i] = c
    return token_to_id, id_to_token

SPECIAL_TOKENS = ['<PAD>', '<SOS>', '<EOS>', '<SPACE>']
_char2id, _id2char = char2id(SPECIAL_TOKENS)

def tokenize(uppercase_sentence):
  """ Tokenize & numericize (i.e. assign integers to each character) the given sentence. """
  tokens = [_char2id[char] if char != ' ' else _char2id['<SPACE>'] for char in list(uppercase_sentence)]
  tokens += [_char2id['<EOS>']]
  return tokens

In [51]:

def tokenize_sentences(sentences):
  """ Iterate through a list of sentences and tokenize them.
  Returns: ( an array of tokens, their lengths )"""
  tokens = []
  lengths = []
  for sentence in sentences:
      # sentence = sentence.translate(str.maketrans('', '', string.punctuation))
      sentence_converted = tokenize(sentence)
      tokens.append(sentence_converted)
      lengths.append(len(sentence_converted))
  return np.array(tokens), np.array(lengths).astype(np.int32)


In [53]:
# Show examples
print("Numericization dict: "+str(_char2id))
print("'TEST' tokenized is "+str(tokenize('TEST'))+" where the 5th token is <EOS>") 

Numericization dict: {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<SPACE>': 3, 'A': 4, 'B': 5, 'C': 6, 'D': 7, 'E': 8, 'F': 9, 'G': 10, 'H': 11, 'I': 12, 'J': 13, 'K': 14, 'L': 15, 'M': 16, 'N': 17, 'O': 18, 'P': 19, 'Q': 20, 'R': 21, 'S': 22, 'T': 23, 'U': 24, 'V': 25, 'W': 26, 'X': 27, 'Y': 28, 'Z': 29}
'TEST' tokenized is [23, 8, 22, 23, 2] where the 5th token is <EOS>


Now we can extract features and target text from the whole LibriSpeech dataset!

In [112]:
# The following code borrowed/modified from https://github.com/30stomercury/Automatic-Speech-Recognition/blob/master/preprocess.py

from glob import glob
import joblib
import logging

def get_texts_and_audio_paths(root_dataset_path):
  """ Iterate through folders in a directory.
  Return: Target texts, paths of correspending audio recordings 
  """
  folders = glob(root_dataset_path+"/**/**")
  texts = []
  audio_path = []
  for path in folders:
    text_path = glob(path+"/*txt")[0]
    f = open(text_path)
    for line in f.readlines():
        line_ = line.split(" ")
        audio_path.append(path+"/"+line_[0]+".flac")
        texts.append(line[len(line_[0])+1:-1].replace("'",""))
  return texts, audio_path

# When number of audios in a set (usually training set) > threshold, divide set into several parts to avoid memory error.
_SAMPLE_THRESHOLD = 30000

def extract_inputs(datasets, datasets_dir, feat_dir):
  """ Iterate through LibriSpeech dataset and save features and texts to 
      binaries under feat_dir. 
  """
  def process_libri_feats(audio_path, cat, k):
      """When number of feats > threshold, divide feature
          into several parts to avoid memory error.
      """
      if len(audio_path) > _SAMPLE_THRESHOLD:
          featlen = []
          n = len(audio_path) // k + 1
          logging.info("Process {} audios...".format(cat))
          for i in range(k):
              feats, featlen_ = files_to_log_mel_spec(audio_path[i*n:(i+1)*n])
              featlen += featlen_
              # save
              joblib.dump(feats, feat_dir+"/{}-feats-{}.pkl".format(cat, i))
              feats = []
      else:
          feats, featlen = files_to_log_mel_spec(audio_path, sample_rate, window_size, window_step, n_filters)
          joblib.dump(feats, feat_dir+"/{}-feats.pkl".format(cat))
      np.save(feat_dir+"/{}-featlen.npy".format(cat), featlen)

  for dataset_name in datasets:
      to_cat = dataset_name
      libri_path = datasets_dir + dataset_name + '/' + 'LibriSpeech' + '/' + dataset_name
      print(libri_path)
      target_texts, audio_paths = get_texts_and_audio_paths(libri_path)

      # Tokenize sentence
      tokens, token_lengths = tokenize_sentences(target_texts)

      # Save tokens and their lengths to files
      np.save(feat_dir+"/{}-{}s.npy".format(to_cat,'char'), tokens)
      np.save(feat_dir+"/{}-{}len.npy".format(to_cat,'char'), token_lengths)

      # Extract and download features (in our case, the Log Mel Spectrograms)
      process_libri_feats(audio_paths, to_cat, len(audio_paths)//_SAMPLE_THRESHOLD)


In [138]:

FEATURES_DIR = './features'
!mkdir -p {FEATURES_DIR}
extract_inputs(DATASETS, DATASET_DIR, FEATURES_DIR)

./data/dev-clean/LibriSpeech/dev-clean


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


ValueError: ignored

In [155]:
!pip install tensorflow_io
import tensorflow_io as tfio

In [177]:
!wget -O /content/data/test.wav https://file-examples-com.github.io/uploads/2017/11/file_example_WAV_1MG.wav

--2021-06-06 22:18:05--  https://file-examples-com.github.io/uploads/2017/11/file_example_WAV_1MG.wav
Resolving file-examples-com.github.io (file-examples-com.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to file-examples-com.github.io (file-examples-com.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1073218 (1.0M) [audio/wav]
Saving to: ‘/content/data/test.wav’

/content/data/test. 100%[===================>]   1.02M  --.-KB/s    in 0.06s   

2021-06-06 22:18:05 (17.0 MB/s) - ‘/content/data/test.wav’ saved [1073218/1073218]



In [204]:

def path_to_audio(path, sample_rate):
    # spectrogram using stft
    audio = tf.io.read_file(path)
    audio, _ = librosa.load(path, sample_rate)
    audio = tf.convert_to_tensor(audio)
    # audio, _ = tf.audio.decode_wav(audio, 1)
    # audio = tf.squeeze(audio, axis=-1)
    # return audio
    stfts = tf.signal.stft(audio, frame_length=window_size_frames, frame_step=window_step_frames, fft_length=1024)
    spectrograms = tf.abs(stfts)

    # Warp the linear scale spectrograms into the mel-scale.
    num_spectrogram_bins = stfts.shape[-1]
    lower_edge_hertz, upper_edge_hertz, num_mel_bins = 80.0, 7600.0, 80
    linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
      num_mel_bins, num_spectrogram_bins, sample_rate, lower_edge_hertz,
      upper_edge_hertz)
    mel_spectrograms = tf.tensordot(spectrograms, linear_to_mel_weight_matrix, 1)
    mel_spectrograms.set_shape(spectrograms.shape[:-1].concatenate(
      linear_to_mel_weight_matrix.shape[-1:]))

    # Compute a stabilized log to get log-magnitude mel-scale spectrograms.
    lmspec = tf.math.log(mel_spectrograms + 1e-6)

    # normalisation
    means = tf.math.reduce_mean(lmspec, 1, keepdims=True)
    stddevs = tf.math.reduce_std(lmspec, 1, keepdims=True)
    lmspec = (lmspec - means) / stddevs
    audio_len = tf.shape(lmspec)[0]
    # padding to 10 seconds
    pad_len = 2754
    paddings = tf.constant([[0, pad_len], [0, 0]])
    lmspec = tf.pad(lmspec, paddings, "CONSTANT")[:pad_len, :]
    return lmspec

path_to_audio('./data/dev-clean/LibriSpeech/dev-clean/1272/128104/1272-128104-0000.flac')

TypeError: ignored

Even though we have the spectrograms of audio recordings and their respective text translations, the Neural Network will not be able to read them in this format. We'll need to define the following variables so that both the encoder and decoder can run:

In [ ]:
num_encoder_tokens = None  #
num_decoder_tokens = None
encoder_input_data = None
decoder_input_data = None
max_encoder_seq_length = None
max_decoder_seq_length = None

decoder_target_data = None

In [ ]:

    # target_token_index = dict([(char, i) for i, char in enumerate(total_found_chars)])

def get_decoder_data():  # populate 2 arrays, offset by 1
    decoder_input_data = np.zeros(
        (num_output, max_decoder_seq_length, num_decoder_tokens), dtype="float32"
    )
    decoder_target_data = np.zeros(
        (num_output, max_decoder_seq_length, num_decoder_tokens), dtype="float32"
    )
    for char in found_chars:
        idx =target_token_index[char]
        array[idx] = 1.0

        # How do we do input/output arrays of diffff length?
        # https://keras.io/examples/nlp/lstm_seq2seq/
        # https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/nlp/ipynb/lstm_seq2seq.ipynb#scrollTo=Tp6YF0oHXgay
        # https://medium.com/deep-learning-with-keras/seq2seq-part-e-encoder-decoder-for-variable-input-output-size-with-teacher-forcing-92c476dd9b0



In [ ]:
# add function to AudioHandler
log_specgrams, \
target_text_chars, \
total_found_chars, \
num_target_text_tokens = extract_features_and_text_targets('./data_test/LibriSpeech/dev-clean/')
# https://www.kdnuggets.com/2017/12/audio-classifier-deep-neural-networks.html

 and Numericize them (i.e. assign integers to each token)

In [ ]:
target_token_index = None  # A dict mapping an integer to each seen character
encoder_input_data = None
decoder_input_data = None  # An array of dicts mapping characters to whether 1 if seen, 0 if not seen
decoder_target_data = None  # An array of dicts mapping characters to whether 1 if seen, 0 if not seen


1272-135031-0002
1272-135031-0003
1272-135031-0018
1272-135031-0019
1272-135031-0011
1272-135031-0022
1272-135031-0020
1272-135031-0007
1272-135031-0006
1272-135031-0009
1272-135031-0014
1272-135031-0010
1272-135031-0013
1272-135031-0017
1272-135031-0021
1272-135031-0000
1272-135031-0005
1272-135031-0023
1272-135031-0016
1272-135031-0008
1272-135031-0012
1272-135031-0001
1272-135031-0015
1272-135031-0004
1272-135031-0024
1272-128104-0005
1272-128104-0008
1272-128104-0014
1272-128104-0011
1272-128104-0001
1272-128104-0013
1272-128104-0012
1272-128104-0004
1272-128104-0007
1272-128104-0002
1272-128104-0003
1272-128104-0000
1272-128104-0009
1272-128104-0006
1272-128104-0010
1272-141231-0011
1272-141231-0018
1272-141231-0030
1272-141231-0013
1272-141231-0012
1272-141231-0010
1272-141231-0019
1272-141231-0022
1272-141231-0006
1272-141231-0032
1272-141231-0001
1272-141231-0025
1272-141231-0027
1272-141231-0026
1272-141231-0015
1272-141231-0003
1272-141231-0024
1272-141231-0028
1272-141231-00

[]

In [ ]:

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
data_path = "./data/LibriSpeech/dev-clean/1272/128104/1272-128104-0000.flac"




In [ ]:
import keras

encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))


<module 'keras.datasets.boston_housing' from '/home/a/.local/lib/python3.8/site-packages/keras/datasets/boston_housing.py'>



Next: build encoder-decoder architecture, based on https://github.com/tensorflow/nmt

Next: plug into RNN, run it on Google Colab. Save it

Next: let user input voice data, and run it through the model

More soucres
https://towardsdatascience.com/recognizing-speech-commands-using-recurrent-neural-networks-with-attention-c2b2ba17c837
https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html
https://jalammar.github.io/visualizing-neural-machine-translation-mechanics-of-seq2seq-models-with-attention/
https://arxiv.org/pdf/1409.0473.pdf


Further Optimizations:

The Context Vector being fixed-length turns out to be a performance bottleneck; given a long input sentences, the the Encoder may not be able to store all of its output in the Context Vector in one timestep.

To solve the bottleneck, we could use the 'attention' mechanism which involves the Decoder selecting from all hidden states provided by the Encoder. For the sake of simplicity we won't be doing this optimization.
